In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import MySQLdb

from BoxScoreParser import BoxScoreParser
from UrlParser import UrlParser
from sklearn import tree
from sklearn.externals.six import StringIO

In [2]:
# MySQL conf
con=MySQLdb.connect(host='link10th.cjwmbrma7nnv.ap-northeast-2.rds.amazonaws.com',user='link10th',passwd='link1234',db='link10th',charset='utf8', use_unicode=True)

# DecisionTree 
path = './ScoreCase/DecisionTree_sample.csv'
df = pd.read_csv(path,index_col=None,header=None,
                 names=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum',
                        'context'])
score = np.array(df.reindex(columns=['1H','2H','3H','4H','5H','6H','7H','8H','9H','10H','11H','12H','sum']))
context = np.array(df.get('context'))
context_sen = [' ',
               '승리하였다.',
               '접전끝에 승리를 하였다.',
               '역전에 성공했다.',
               '완승을 하였다.',
               '무승부로 끝이났다.',
               '영봉승을 하였다.']

clf = tree.DecisionTreeClassifier(criterion='entropy')
clf = clf.fit(score,context)
clf.predict(np.array(score))

array([1, 1, 1, ..., 1, 3, 3])

In [18]:
urlParser = UrlParser('20160520')

for url in urlParser.urlList:
    # articleDate
    bxsParser=BoxScoreParser(url)
    date=bxsParser.date[5:7]+u"월"+bxsParser.date[8:10]+u"일"+bxsParser.date[10:]

    # articleHead
    head=u'at ascore vs ht hscore'
    head=head.replace('at',bxsParser.a_team)
    head=head.replace('ht',bxsParser.h_team)
    head=head.replace('hscore',str(bxsParser.h_score[12]))
    head=head.replace('ascore',str(bxsParser.a_score[12]))

    # articlePre
    pre=u'at는 date stadium경기장에서 열린 ht와의 경기에서 ascore-hscore으로 context'
    pre=pre.replace('ht',bxsParser.h_team)
    pre=pre.replace('date',bxsParser.date)
    pre=pre.replace('stadium',bxsParser.stadium)
    pre=pre.replace('at',bxsParser.a_team)
    pre=pre.replace('hscore',str(bxsParser.h_score[12]))
    pre=pre.replace('ascore',str(bxsParser.a_score[12]))
    
    # context
    #'context' -> exception_case(무승부)
    if bxsParser.h_score[12]==bxsParser.a_score[12]:
        pre=pre.replace('context',context_sen[5].decode('utf-8'))    
    #'context' -> exception_case(영봉승)
    elif bxsParser.h_score[12]==0 or bxsParser.a_score[12]==0:
        pre=pre.replace('context',context_sen[6].decode('utf-8'))    
    else:
        pre=pre.replace('context',context_sen[clf.predict([x-y for x,y in zip(bxsParser.h_score[:13],bxsParser.a_score[:13])])].decode('utf-8'))
    #HOME팀이 이긴경우 승리->패배
    if bxsParser.a_score[12]<bxsParser.h_score[12]:
        pre=pre.replace(u'승리',u'패배')
        pre=pre.replace(u'성공',u'실패')
        pre=pre.replace(u'완승',u'완패')
        pre=pre.replace(u'영봉승',u'영봉패')

    # charset
    date.encode('utf8')
    head.encode('utf8')
    pre.encode('utf8')
    url.encode('utf8')

    #교정부분
    
    #etc
    if bxsParser.a_score[12]>=bxsParser.h_score[12]:
        winTeam=bxsParser.a_team
        loseTeam=bxsParser.h_team
    else:
        winTeam=bxsParser.h_team
        loseTeam=bxsParser.a_team
    
    cursor=con.cursor()
    cursor.execute('INSERT INTO Article(date,head,pre,url,winTeam,loseTeam) VALUES (\'%s\',\'%s\',\'%s\',\'%s\',\'%s\',\'%s\')'%(date,head,pre,url,winTeam,loseTeam))

    # MySQL disconnect
    con.commit()
    cursor.close()
con.close()

/home/gohyunyoung98/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/gohyunyoung98/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:32: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
/home/gohyunyoung98/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/home/gohyunyoung98/Enthought/Canopy_64bit/User/lib/python2.